In [1]:
import glob
folder = r'../../BLUED/location_001_dataset_001/'
allFiles = glob.glob(folder+"location_001_matlab_*.mat")
print(allFiles)

['../../BLUED/location_001_dataset_001\\location_001_matlab_1.mat', '../../BLUED/location_001_dataset_001\\location_001_matlab_2.mat', '../../BLUED/location_001_dataset_001\\location_001_matlab_3.mat', '../../BLUED/location_001_dataset_001\\location_001_matlab_4.mat']


In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import scipy.io as scio
import glob

In [3]:
total = pd.DataFrame()
for each_mat in allFiles:
    data_train=scio.loadmat(each_mat)
    print(each_mat) #输出每个mat文件
    data = data_train.get('data')  #读取每个mat文件的data数据库
    #data_train=scio.loadmat(r'BLUED/location_001_dataset_001/location_001_matlab_1.mat')
    #读取data数据库的有用功率、视在功率
    t_power = pd.DataFrame(data[0][0][0],columns=['t_power'])
    Qa = pd.DataFrame(data[0][0][1],index=['Qa'])
    Qb = pd.DataFrame(data[0][0][2],index=['Qb'])
    Pa = pd.DataFrame(data[0][0][3],index=['Pa'])
    Pb = pd.DataFrame(data[0][0][4],index=['Pb'])
    StartDate = data[0][0][5]
    StartTime = data[0][0][6]
    print(StartTime)
    #wdw_size = data[0][0][7]
    #window_shift = data[0][0][8]
    ##处理数据行，合并成dataframe形式
    df_m = pd.concat([Qa,Qb,Pa,Pb])
    df2 = pd.DataFrame(df_m.values.T, index=df_m.columns, columns=df_m.index)
    df2 = pd.concat([t_power,df2],axis=1)
    #print(df2)
    total = pd.concat([total,df2])
    df2 = []
total.reset_index(drop=True, inplace=True) ##重新整理index
total

../../BLUED/location_001_dataset_001\location_001_matlab_1.mat
[[array([[array(['11:58:32.6234999999996440291'], dtype='<U28')]],
      dtype=object)]]
../../BLUED/location_001_dataset_001\location_001_matlab_2.mat
[[array([[array(['14:44:05.7734999999976968957'], dtype='<U28')]],
      dtype=object)]]
../../BLUED/location_001_dataset_001\location_001_matlab_3.mat
[[array([[array(['17:25:27.6734999999957990133'], dtype='<U28')]],
      dtype=object)]]
../../BLUED/location_001_dataset_001\location_001_matlab_4.mat
[[array([[array(['20:06:24.0234999999939061394'], dtype='<U28')]],
      dtype=object)]]


,t_power,Qa,Qb,Pa,Pb
0,734796.498989,118.251216,161.176490,40.517450,198.107066
1,734796.498989,118.141667,160.610125,40.504628,197.147953
2,734796.498989,117.672809,160.958658,40.046103,199.141009
3,734796.498989,117.687076,161.179989,40.459411,202.366780
4,734796.498989,117.772242,162.074567,40.438556,207.740681
...,...,...,...,...,...
2334801,734796.949375,48.634544,172.812957,165.906496,809.500995
2334802,734796.949375,48.304674,172.931443,165.283941,803.991071
2334803,734796.949376,48.117450,172.608530,165.214087,801.954426
2334804,734796.949376,48.796998,173.318307,165.914167,808.454201


In [4]:
N=60
total = total.groupby(total.index // N).sum()
total = total/N
total

,t_power,Qa,Qb,Pa,Pb
0,734796.498994,117.689789,161.055841,40.443050,201.332245
1,734796.499006,117.696156,161.099501,40.464056,201.673950
2,734796.499018,117.700292,161.050049,40.509447,200.967639
3,734796.499029,117.597273,161.219237,40.419981,201.068677
4,734796.499041,117.923122,161.826438,40.544850,201.311852
...,...,...,...,...,...
38909,734796.949331,48.253608,172.421885,165.253601,803.210960
38910,734796.949342,48.279151,171.778576,165.508748,806.948848
38911,734796.949354,48.270904,169.824527,165.419797,819.775823
38912,734796.949365,48.395249,171.885241,165.569094,804.483656


In [5]:
#初始化datetime类型，
from datetime import datetime
beg_time = datetime(2011,10,20,11,58,32,623499)
end_time = datetime(2011,10,20,22,45,29,893499)
print(beg_time)
print(end_time)
print((end_time-beg_time).seconds)
from datetime import timedelta
td = timedelta(days=0, seconds=0, microseconds=beg_time.microsecond, milliseconds=0, minutes=0, hours=0, weeks=0)
beg_time = beg_time - td
beg_time.timestamp()
import datetime
delta=datetime.timedelta(seconds=1)
print(beg_time)


2011-10-20 11:58:32.623499
2011-10-20 22:45:29.893499
38817
2011-10-20 11:58:32


In [6]:
total['time']=beg_time+total.index*delta

In [7]:
time = total['time']
total.drop(labels=['time'],axis=1)
total.insert(0,'Time',time)
total

,Time,t_power,Qa,Qb,Pa,Pb,time
0,2011-10-20 11:58:32,734796.498994,117.689789,161.055841,40.443050,201.332245,2011-10-20 11:58:32
1,2011-10-20 11:58:33,734796.499006,117.696156,161.099501,40.464056,201.673950,2011-10-20 11:58:33
2,2011-10-20 11:58:34,734796.499018,117.700292,161.050049,40.509447,200.967639,2011-10-20 11:58:34
3,2011-10-20 11:58:35,734796.499029,117.597273,161.219237,40.419981,201.068677,2011-10-20 11:58:35
4,2011-10-20 11:58:36,734796.499041,117.923122,161.826438,40.544850,201.311852,2011-10-20 11:58:36
...,...,...,...,...,...,...,...
38909,2011-10-20 22:47:01,734796.949331,48.253608,172.421885,165.253601,803.210960,2011-10-20 22:47:01
38910,2011-10-20 22:47:02,734796.949342,48.279151,171.778576,165.508748,806.948848,2011-10-20 22:47:02
38911,2011-10-20 22:47:03,734796.949354,48.270904,169.824527,165.419797,819.775823,2011-10-20 22:47:03
38912,2011-10-20 22:47:04,734796.949365,48.395249,171.885241,165.569094,804.483656,2011-10-20 22:47:04


In [8]:
total['apparentA'] = pow((pow(total['Qa'],2)+pow(total['Pa'],2)),1/2)
total['apparentB'] = pow((pow(total['Qb'],2)+pow(total['Pb'],2)),1/2)
total

,Time,t_power,Qa,Qb,Pa,Pb,time,apparentA,apparentB
0,2011-10-20 11:58:32,734796.498994,117.689789,161.055841,40.443050,201.332245,2011-10-20 11:58:32,124.444875,257.824857
1,2011-10-20 11:58:33,734796.499006,117.696156,161.099501,40.464056,201.673950,2011-10-20 11:58:33,124.457724,258.119026
2,2011-10-20 11:58:34,734796.499018,117.700292,161.050049,40.509447,200.967639,2011-10-20 11:58:34,124.476399,257.536619
3,2011-10-20 11:58:35,734796.499029,117.597273,161.219237,40.419981,201.068677,2011-10-20 11:58:35,124.349883,257.721274
4,2011-10-20 11:58:36,734796.499041,117.923122,161.826438,40.544850,201.311852,2011-10-20 11:58:36,124.698627,258.291033
...,...,...,...,...,...,...,...,...,...
38909,2011-10-20 22:47:01,734796.949331,48.253608,172.421885,165.253601,803.210960,2011-10-20 22:47:01,172.154475,821.509070
38910,2011-10-20 22:47:02,734796.949342,48.279151,171.778576,165.508748,806.948848,2011-10-20 22:47:02,172.406560,825.029892
38911,2011-10-20 22:47:03,734796.949354,48.270904,169.824527,165.419797,819.775823,2011-10-20 22:47:03,172.318860,837.181444
38912,2011-10-20 22:47:04,734796.949365,48.395249,171.885241,165.569094,804.483656,2011-10-20 22:47:04,172.497028,822.641166


In [9]:
total['Time_num'] = pd.to_numeric(total['Time'])

In [ ]:
from os.path import join
aggregate_data = []
for row in total.itertuples():
    print(row)
    aggregate_data.append([float(getattr(row, 'Time_num')),float(getattr(row, 'Pa'))])
print(aggregate_data)
np.save(join(folder, 'main.npy'), aggregate_data)
print('mat finished!')